In [1]:
import numpy as np
def simplex(c,A,b):
    # 当输入的c,A,b有缺失值时，输出错误原因，函数执行结束
    if c.shape[0] != A.shape[1]:
        print("A和C形状不匹配")
        return 0
    if b.shape[0] != A.shape[0]:
        print("A和b形状不匹配")
        return 0
    num=A.shape[1]-A.shape[0]
    N_indexs=np.arange(0,num) # 非基变量的下标索引
    B_indexs=np.arange(num,A.shape[1]) # 基变量的下标索引
    N=A[:,N_indexs] # 初始的N
    B=A[:,B_indexs] # 初始的B
    c_N=c[N_indexs,:]
    c_B=c[B_indexs,:]
    x_B = np.matmul(np.linalg.inv(B), b)
    i=0
    while 1:
        # 判断r值是否>=0，若r的每一维都>=0，则满足最优化条件。若r中有值<0，则继续寻找最优点。
        R=(c_N.T - np.matmul(np.matmul(c_B.T, np.linalg.inv(B)), N)).flatten()
        if all(R >= 0):
            print("##########################找到最优点##########################")
            print("非基变量的索引为",N_indexs)
            print("基变量的索引为",B_indexs)
            print("最优点为",x_B.flatten())
            print("最优值为",z.flatten()[0])
            return 0
        else:
            i=i+1
            print("step",i)
            # 找到入基变量
            N_in_index = np.argmin(R)  # 找到入基变量的下标
            N_in=N[:,N_in_index] # 入基变量对应的Aq
            # 找到出基变量
            negd=np.matmul(np.linalg.inv(B),N_in)
            y=x_B.flatten()/negd
            # 设置一个初值索引index，需满足negd[index]>0
            for i in range(len(y)):
                if negd[i]>0:
                    index=i
                    break
            # 找到y最小且negd>0的位置
            for i in range(len(y)):
                if negd[i]<=0:
                    continue
                elif y[i]<y[index]:
                    index=i
            B_out_index=index # 找到出基变量的下标
            B_out=B[:,B_out_index] # 出基变量对应的Aq
            #出入基变量交换位置
            in_index=N_indexs[N_in_index]
            out_index=B_indexs[B_out_index]
            temp=N_indexs[N_in_index]
            N_indexs[N_in_index]=B_indexs[B_out_index]
            B_indexs[B_out_index]=temp
            N[:,N_in_index]=B_out
            B[:,B_out_index]=N_in
            c_N = c[N_indexs, :]
            c_B = c[B_indexs, :]
            N = A[:, N_indexs]
            B = A[:, B_indexs]
            x_B = np.matmul(np.linalg.inv(B), b)
            z = np.matmul(c_B.T, x_B)
            print("出基变量索引",out_index)
            print("入基变量索引",in_index)
            print("值为",z.flatten()[0])

c = np.array([-5, -1, 0,0]).reshape(-1, 1)
A = np.array([[1, 1, 1, 0], [2, 0.5, 0, 1]])
b = np.array([5,8]).reshape(-1, 1)
simplex(c,A,b)

step 1
出基变量索引 3
入基变量索引 0
值为 -20.0
##########################找到最优点##########################
非基变量的索引为 [3 1]
基变量的索引为 [2 0]
最优点为 [1. 4.]
最优值为 -20.0


0

In [2]:
import numpy as np
def pivot(d,bn):
    l = list(d[0][:-2])
    jnum = l.index(max(l)) #转入编号
    m = []
    for i in range(bn):
        if d[i][jnum] == 0:
            m.append(0.)
        else:
            m.append(d[i][-1]/d[i][jnum])
    inum = m.index(min([x for x in m[1:] if x!=0]))  #转出下标
    s[inum-1] = jnum
    r = d[inum][jnum]
    d[inum] /= r
    for i in [x for x in range(bn) if x !=inum]:
        r = d[i][jnum]
        d[i] -= r * d[inum]  
#定义基变量函数
def solve(d,bn):
    flag = True
    while flag:
        if max(list(d[0][:-1])) <= 0: #直至所有系数小于等于0
            flag = False
        else:
            pivot(d,bn)            
def printSol(d,cn):
    for i in range(cn - 1):
        if i in s:
            print("x"+str(i)+"=%.2f" % d[s.index(i)+1][-1])
        else:
            print("x"+str(i)+"=0.00")
    print("objective is %.2f"%(-d[0][-1]))
d = np.array([[5,1,0,0,0],[1,1,1,0,5],[2,1/2,0,1,8]])
(bn,cn) = d.shape
s = list(range(cn-bn,cn-1)) #基变量列表
solve(d,bn)
printSol(d,cn)



x0=4.00
x1=0.00
x2=1.00
x3=0.00
objective is 20.00


In [3]:
import numpy as np
def Interior_Point(c,A,b):
    # 当输入的c,A,b有缺失值时，输出错误原因，函数执行结束
    if c.shape[0] != A.shape[1]:
        print("A和C形状不匹配")
        return 0
    if b.shape[0] != A.shape[0]:
        print("A和b形状不匹配")
        return 0
    # 初值的设置
    x=np.ones((A.shape[1],1)) # 将x的初值设为1
    v=np.ones((b.shape[0],1))  # 将v的初值设为1
    lam=np.ones((x.shape[0],1))  # 将lam的初值设为1
    one=np.ones((x.shape[0],1))
    mu=1 # 将mu的初值设为1
    n=A.shape[1]
    x_=np.diag(x.flatten()) # 将x转换为对角矩阵
    lam_=np.diag(lam.flatten()) # 将lam转换为对角矩阵
    # 初始的F，r=F
    r1=np.matmul(A,x)-b
    r2=np.matmul(np.matmul(x_,lam_),one)-mu*one
    r3=np.matmul(A.T,v)+c-lam
    r=np.vstack((r1,r2,r3))
    F=r
    # 求得r1、r2、r3的初始范数
    n1=np.linalg.norm(r1)
    n2=np.linalg.norm(r2)
    n3=np.linalg.norm(r3)
    # nablaF中零矩阵和单位阵的设置
    zero11=np.zeros((A.shape[0],x.shape[0]))
    zero12=np.zeros((A.shape[0],A.shape[0]))
    zero22=np.zeros((x.shape[0],A.shape[0]))
    zero33=np.zeros((A.shape[1],A.shape[1]))
    one31=np.eye(A.shape[1])
    tol=1e-8 # 设置最优条件的容忍度
    t=1
    alpha = 0.5
    while max(n1,n2,n3)>tol:
        print("-----------------step",t,"-----------------")
        # F的Jacobian矩阵
        nablaF = np.vstack((np.hstack((zero11, zero12, A))
                            , np.hstack((x_, zero22, lam_))
                            , np.hstack((-one31, A.T, zero33))))
        # F+nablaF@delta=0,解方程nablaF@delta=-r
        delta = np.linalg.solve(nablaF, -r)  # 解方程，求出delta的值
        delta_lam = delta[0:lam.shape[0], :]
        delta_v = delta[lam.shape[0]:lam.shape[0] + v.shape[0], :]
        delta_x = delta[lam.shape[0] + v.shape[0]:, :]
        # 更新lam、v、x、mu
        alpha=Alpha(c,A,b,lam,v,x,alpha,delta_lam,delta_v,delta_x)
        lam=lam+alpha*delta_lam
        v=v+alpha*delta_v
        x=x+alpha*delta_x
        x_ = np.diag(x.flatten())  # 将x转换为对角矩阵
        lam_ = np.diag(lam.flatten())  # 将lam转换为对角矩阵
        mu=(0.1/n)*np.dot(lam.flatten(),x.flatten()) #更新mu的值
        # 计算更新后的F
        r1 = np.matmul(A, x) - b
        r2 = np.matmul(np.matmul(x_, lam_), one) - mu * one
        r3 = np.matmul(A.T, v) + c - lam
        r = np.vstack((r1, r2, r3))
        F = r
        # 计算更新后F的范数
        n1 = np.linalg.norm(r1)
        n2 = np.linalg.norm(r2)
        n3 = np.linalg.norm(r3)
        t=t+1
        print("x的取值",x.flatten())
        print("v的取值",v.flatten())
        print("lam的取值",lam.flatten())
        print("mu的取值",mu)
        print("alpha的取值",alpha)
        z=(c.T @ x).flatten()[0]
        print("值为",z)
    print("##########################找到最优点##########################")
    print("x的取值",x.flatten())
    print('最优值为',z)

# 寻找alpha
def Alpha(c,A,b,lam,v,x,alpha,delta_lam,delta_v,delta_x):
    alpha_x=[]
    alpha_lam=[]
    for i in range(x.shape[0]):
        if delta_x.flatten()[i]<0:
            alpha_x.append(x.flatten()[i]/-delta_x.flatten()[i])
        if delta_lam.flatten()[i]<0:
            alpha_lam.append(lam.flatten()[i]/-delta_lam.flatten()[i])
    if len(alpha_x)==0 and len(alpha_lam)==0:
        return alpha
    else:
        alpha_x.append(np.inf)
        alpha_lam.append(np.inf)
        alpha_x = np.array(alpha_x)
        alpha_lam= np.array(alpha_lam)
        alpha_max = min(np.min(alpha_x), np.min(alpha_lam))
        alpha_k = min(1,0.99*alpha_max)
    return alpha_k

c = np.array([-5, -1, 0,0]).reshape(-1, 1)
A = np.array([[1, 1, 1, 0], [2, 0.5, 0, 1]])
b = np.array([5, 8]).reshape(-1, 1)
Interior_Point(c,A,b)

-----------------step 1 -----------------
x的取值 [1.99   1.0225 0.8425 0.9325]
v的取值 [1.1575 1.0675]
lam的取值 [0.01   0.9775 1.1575 1.0675]
mu的取值 0.07475078125
alpha的取值 0.4275
值为 -10.9725
-----------------step 2 -----------------
x的取值 [3.20879395 0.6297997  0.60221202 0.009325  ]
v的取值 [0.94082119 1.61918231]
lam的取值 [0.01797731 0.89021093 0.94082119 1.61918231]
mu的取值 0.030000319317640158
alpha的取值 0.5116206718282758
值为 -16.67376946490082
-----------------step 3 -----------------
x的取值 [3.85861428 0.49249655 0.63858577 0.0133405 ]
v的取值 [0.00940821 2.49050221]
lam的取值 [0.00586775 0.25723517 0.00940821 2.49050221]
mu的取值 0.004714032760468829
alpha的取值 0.981574544421685
值为 -19.78556794032732
-----------------step 4 -----------------
x的取值 [3.99200059e+00 2.83066232e-02 9.79692788e-01 1.84551131e-03]
v的取值 [2.35649726e-03 2.49933118e+00]
lam的取值 [1.01885150e-03 2.52022086e-01 2.35649726e-03 2.49933118e+00]
mu的取值 0.0004530584332237983
alpha的取值 1
值为 -19.988309565940718
-----------------step 5 -------------